This file is meant to pre-process the training and validation data from the directories of tiny-imagenet into npz files which can be uploaded to Google Drive to be used for training on Colab. It saves two files on the disk, botch of which contain two numpy arrays, one containing the input samples of the dataset in form of numpy arrays and the. other the class label in form of one-hot encoded vectors.

In [1]:
import numpy as np
from PIL import Image

In [2]:
#THIS BLOCK IS IMPORTANT

def importClasses():
    classPath = "./tiny-imagenet-200/wnids.txt"
    return open(classPath).read().splitlines()

trainClasses = importClasses()

def createDict(trainClasses):
    #Mapping from class name to class Number
    classDict = {}
    for i in range(len(trainClasses)):
        classDict[trainClasses[i]] = i
    return classDict

classNumDict = createDict(trainClasses)

def getImgAsNP(path):
    try:
        img = Image.open(path)
    except IOError:
        return
    img = img.convert("RGB")
    img = np.asarray(img)
    return img
    


def transToOneHot(numClass, dims = 200):
    vec = np.zeros(dims)
    vec[numClass] = 1
    return vec


def randomizeOrder(inArr, outArr, n):
    idx = np.array(range(n))
    np.random.shuffle(idx)
    inList = []
    outList = []
    for i in range(n):
        inList.append(inArr[idx[i]])
        outList.append(outArr[idx[i]])
    return inList, outList




def getOutputArray():
    outputArray = []
    outputTextPath = "./tiny-imagenet-200/val/val_annotations.txt"
    textFile = open(outputTextPath).read().split()
    for i in range(10000):
        outputArray.append(transToOneHot(classNumDict[textFile[6*i+1]], dims = 200))
    return outputArray

def getValSample():
    inputArray = []
    outputArray = getOutputArray()
    
    
    for i in range(10000):
        path = "./tiny-imagenet-200/val/images/val_" + str(i) + ".JPEG"
        img = getImgAsNP(path)
        inputArray.append(img)
        
    return inputArray, outputArray

In [3]:
def generateImgsWOAug(path, outClassNum):
    #Takes in path of Image and generates 5 patches and their mirror images and returns them as a list of np arrays and an array of one hot encoded output
    try:
        img = Image.open(path)
    except IOError:
        return
    img = img.convert("RGB")
    img = np.asarray(img)
    #print("This is for debugging:" + img.shape)
    arr = [img[:, :, :]]
    #arr = arr + [np.flip(arr[0], 1),np.flip(arr[1], 1), np.flip(arr[2], 1), np.flip(arr[3], 1), np.flip(arr[4], 1)]
    #pcaArr = []
    #for i in range(len(arr)):
    #    pcaArr.append(pcaAug(arr[i]))
    inputArr = arr# + pcaArr
    oneHot = transToOneHot(outClassNum, dims = 200)
    outArr = []
    for i in range(len(inputArr)):
        outArr.append(oneHot)
    return inputArr, outArr


def getTrainingSampleWOAug():
    inputArray = []
    outputArray = []
    n = 500
    
    for outClassNum in range(len(trainClasses)):
        for i in range(500):
            path = "./tiny-imagenet-200/train/" + str(trainClasses[outClassNum]) + "/images/" + str(trainClasses[outClassNum]) + "_" + str(i) + ".JPEG"
            inArr, outArr = generateImgsWOAug(path, outClassNum)
            inputArray = inputArray + inArr
            outputArray = outputArray + outArr
    inputArray, outputArray = randomizeOrder(inputArray, outputArray, n*len(trainClasses))
    return inputArray, outputArray

In [8]:
inputTrain, outputTrain = getTrainingSampleWOAug()

In [9]:
inTrain = np.array(inputTrain)
outTrain = np.array(outputTrain)

In [23]:
inTrain.mean()

112.69858580973307

In [24]:
inTrain.std()

70.93752549462194

In [10]:
inTrain.shape

(100000, 64, 64, 3)

In [21]:
np.savez('trainDataComplete.npz', inputTrain, outputTrain)

In [12]:
inputVal, outputVal = getValSample()

In [13]:
inVal = np.array(inputVal)
outVal = np.array(outputVal)

In [14]:
inVal.shape

(10000, 64, 64, 3)

In [15]:
outVal.shape

(10000, 200)

In [22]:
np.savez('valDataComplete.npz', inputVal, outputVal)

Everything after this needs to be deleted

In [ ]:
def normalise(X):
    rMean = 122.46267559570313
    gMean = 114.25840612792969
    bMean = 101.3746757055664
    rStd = 70.63153376175029
    gStd = 68.6114437011529
    bStd = 71.93088608369885
      

In [7]:
rCh = np.array(inTrain[:,:,:,0])
gCh = np.array(inTrain[:,:,:,1])
bCh = np.array(inTrain[:,:,:,2])

In [9]:
rMean = rCh.mean()
gMean = gCh.mean()
bMean = bCh.mean()

In [10]:
rStd = rCh.std()
gStd = gCh.std()
bStd = bCh.std()
print(rMean)
print(gMean)
print(bMean)
print(rStd)
print(gStd)
print(bStd)

122.46267559570313
114.25840612792969
101.3746757055664
70.63153376175029
68.6114437011529
71.93088608369885


In [11]:
rCh = (rCh - rMean)/rStd
gCh = (gCh - gMean)/gStd
bCh = (bCh - bMean)/bStd

In [12]:
print(rCh.mean())
print(gCh.mean())
print(bCh.mean())
print(rCh.std())
print(gCh.std())
print(bCh.std())


-1.1143308498162697e-16
-2.1126433935592102e-17
4.0552006197458465e-17
1.0000000000000053
0.9999999999999973
0.9999999999999977


In [13]:
inTrain = np.array([rCh, gCh, bCh])
inTrain = np.moveaxis(inTrain, 0, -1)

In [19]:
inTrain.shape

(100000, 64, 64, 3)

In [174]:
rChVal = np.array(inVal[:,:,:,0])
gChVal = np.array(inVal[:,:,:,1])
bChVal = np.array(inVal[:,:,:,2])

rChVal = (rChVal - rMean)/rStd
gChVal = (gChVal - gMean)/gStd
bChVal = (bChVal - bMean)/bStd

inVal = np.array([rChVal, gChVal, bChVal])
print(inVal.shape)
inVal = np.moveaxis(inVal, 0, -1)
inVal.shape

(3, 10000, 64, 64)


(10000, 64, 64, 3)

In [177]:
inVal[:,:,:,0].mean()

-4.226378374953798e-05

In [178]:
inTrain.shape

(20000, 64, 64, 3)

In [169]:
inVal.shape

(20000, 64, 64, 3)

In [170]:
outVal.shape

(10000, 200)

In [179]:
intes = np.load('valDataNS.npz')
x = intes['arr_0']
y = intes['arr_1']

In [180]:
x.shape

(10000, 64, 64, 3)

In [181]:
y.shape

(10000, 200)